# Part 3: Data Analytics

## Step 0

> 0. Load both the csv file from **Part 1** `pr.data.0.current` and the json file from **Part 2** as dataframes ([Spark](https://spark.apache.org/docs/1.6.1/api/java/org/apache/spark/sql/DataFrame.html), [Pyspark](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html), [Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), [Koalas](https://koalas.readthedocs.io/en/latest/), etc).

In [45]:
import pandas as pd
import requests

# Data sources (use your signed URLs or correct paths)
SERIES_DATA = "https://rearc-quest-bucket-abhijaya-2025.s3.eu-north-1.amazonaws.com/pr.data.1.AllData"
POPULATION_DATA = "https://rearc-quest-bucket-abhijaya-2025.s3.eu-north-1.amazonaws.com/population.json"

# -----------------------------
# Download and load series data
print("Downloading series data...")
series = pd.read_csv(SERIES_DATA, delimiter="\t")

# Fix: strip whitespace from column names so 'series_id' is correct
series.columns = series.columns.str.strip()

print(f"Series DataFrame shape: {series.shape}")
print("Series columns:", series.columns)
print(series.head())

# -----------------------------
# Download and load population data
print("\nDownloading population data JSON...")
r = requests.get(POPULATION_DATA)
print("Status code:", r.status_code)
if r.status_code != 200:
    print("Error fetching population.json, response preview:")
    print(r.text[:500])
    r.raise_for_status()

population = pd.json_normalize(r.json(), record_path="data")
print(f"Population DataFrame shape: {population.shape}")
print(population.head())

# -----------------------------
# Filter specific series: series IDs containing "PRS30000632" and period "Q01"
specific_series = series.loc[
    series["series_id"].str.contains("PRS30000632", case=False)
]
specific_series = specific_series[specific_series["period"].str.contains("Q01", case=False)]

print("\nFiltered series shape:", specific_series.shape)
print(specific_series.head())

# -----------------------------
# Filter population data for relevant years 2013-2018
population["year"] = population["year"].astype(int)
population_filtered = population[(population["year"] >= 2013) & (population["year"] <= 2018)]

print("\nFiltered population shape:", population_filtered.shape)
print(population_filtered.head())

# -----------------------------
# Merge filtered series with population on 'year'
result = pd.merge(
    specific_series,
    population_filtered,
    left_on="year",
    right_on="year",
    how="left"
)

# Select and order columns for report
result = result[["series_id", "year", "period", "value", "population"]]

# -----------------------------
# Display final merged DataFrame
print("\nMerged result shape:", result.shape)
print(result.head(10))


Series DataFrame shape: (75659, 5)
Series columns: Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object')
           series_id  year period  value footnote_codes
0  PRS30006011        1988    Q01    1.9            NaN
1  PRS30006011        1988    Q02    2.2            NaN
2  PRS30006011        1988    Q03    1.9            NaN
3  PRS30006011        1988    Q04    1.1            NaN
4  PRS30006011        1988    Q05    1.8            NaN

Status code: 200
Population DataFrame shape: (6, 2)
   year  population
0  2013   316128839
1  2014   318857056
2  2015   321418820
3  2016   323127513
4  2017   325719178

Filtered series shape: (0, 5)
Empty DataFrame
Columns: [series_id, year, period, value, footnote_codes]
Index: []

Filtered population shape: (6, 2)
   year  population
0  2013   316128839
1  2014   318857056
2  2015   321418820
3  2016   323127513
4  2017   325719178

Merged result shape: (0, 5)
Empty DataFrame
Columns: [series_id, year, period, value, p

## Step 1

> 1. Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the US population across the years [2013, 2018] inclusive.

In [46]:
# Filter years 2013–2018
population_stats = population[(population["year"] >= 2013) & (population["year"] <= 2018)]

# Calculate mean and standard deviation
mean_pop = population_stats["population"].mean()
std_pop = population_stats["population"].std()

print(f"Mean US population 2013-2018: {mean_pop:,.0f}")
print(f"Stddev US population 2013-2018: {std_pop:,.0f}")


Mean US population 2013-2018: 322,069,808
Stddev US population 2013-2018: 4,158,441


## Step 2

> Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year.

In [47]:
# Remove columns names whitespace
series.rename(columns={"series_id        ": "series_id"}, inplace=True)
series.rename(columns={"       value": "value"}, inplace=True)
# Generate report
max_value_series = series.groupby(["series_id", "year"], as_index=False)["value"].agg("sum")
max_value_series = max_value_series.sort_values("value", ascending=False).drop_duplicates("series_id", keep="first").sort_index().reset_index(drop=True)
max_value_series

,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1989,722.957
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.400
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2024,860.838
279,PRS88003201,1974,61.500
280,PRS88003202,1974,70.700


## Step 3

> Using both dataframes from Part 1 and Part 2, generate a report that will provide the `value` for `series_id = PRS30006032` and `period = Q01` and the `population` for that given year (if available in the population dataset)

In [48]:
print(series["period"].unique())


['Q01' 'Q02' 'Q03' 'Q04' 'Q05']


In [49]:
print(series["series_id"].unique())


['PRS30006011      ' 'PRS30006012      ' 'PRS30006013      '
 'PRS30006021      ' 'PRS30006022      ' 'PRS30006023      '
 'PRS30006031      ' 'PRS30006032      ' 'PRS30006033      '
 'PRS30006061      ' 'PRS30006062      ' 'PRS30006063      '
 'PRS30006081      ' 'PRS30006082      ' 'PRS30006083      '
 'PRS30006091      ' 'PRS30006092      ' 'PRS30006093      '
 'PRS30006101      ' 'PRS30006102      ' 'PRS30006103      '
 'PRS30006111      ' 'PRS30006112      ' 'PRS30006113      '
 'PRS30006131      ' 'PRS30006132      ' 'PRS30006133      '
 'PRS30006151      ' 'PRS30006152      ' 'PRS30006153      '
 'PRS30006161      ' 'PRS30006162      ' 'PRS30006163      '
 'PRS30006171      ' 'PRS30006172      ' 'PRS30006173      '
 'PRS30006211      ' 'PRS30006212      ' 'PRS30006213      '
 'PRS30006221      ' 'PRS30006222      ' 'PRS30006223      '
 'PRS30006231      ' 'PRS30006232      ' 'PRS30006233      '
 'PRS31006011      ' 'PRS31006012      ' 'PRS31006013      '
 'PRS31006021      ' 'PR

In [58]:
# -----------------------------
# Filter specific series: e.g., series IDs containing "PRS30000632" and period "Q01"
specific_series = series.loc[
    series["series_id"].str.contains("PRS30000632", case=False)
]
specific_series = specific_series[
    specific_series["period"].str.contains("Q01", case=False)
]

print("\nInitial filtered specific_series shape:", specific_series.shape)
print(specific_series.head())

# -----------------------------
# Filter specific_series to only years 2013–2018 to match population data
specific_series = specific_series[
    (specific_series["year"] >= 2013) & (specific_series["year"] <= 2018)
]

print("\nFiltered specific_series shape (2013-2018):", specific_series.shape)
print(specific_series.head())

# -----------------------------
# Ensure population['year'] is int
population["year"] = population["year"].astype(int)

# -----------------------------
# Merge filtered series with population data on 'year'
result = pd.merge(
    specific_series,
    population,
    left_on="year",
    right_on="year",
    how="left"
)

# -----------------------------
# Select and order columns for report
result = result[["series_id", "year", "period", "value", "population"]]

# -



Initial filtered specific_series shape: (0, 5)
Empty DataFrame
Columns: [series_id, year, period, value, footnote_codes]
Index: []

Filtered specific_series shape (2013-2018): (0, 5)
Empty DataFrame
Columns: [series_id, year, period, value, footnote_codes]
Index: []


In [56]:
mask = (
    (series["series_id"] == "PRS30006011") &
    (series["year"] >= 2013) &
    (series["year"] <= 2018)
)
print(series.loc[mask, ["year", "period"]].sort_values("year"))


Empty DataFrame
Columns: [year, period]
Index: []


## Step 4

> Submit your analysis, your queries, and the outcome of the reports as a [.ipynb](https://github.com/rearc-data/quest#:~:text=Submit%20your%20analysis%2C%20your%20queries%2C%20and%20the%20outcome%20of%20the%20reports%20as%20a%20.ipynb%20file.) file.

`data_analytics.ipynb` is located in the `part3` directory.